In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd

p = ESMLProject() # Your project settings - knows Azure services, networking, lakedesign, identity mgmt, security, etc
p.inference_mode = False

p.active_model = 11
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.connect_to_lake()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = TRUE 
 - Found settings in the ESML AutoLake  [active_in_folder.json,active_scoring_in_folder.json], to override ArgParse/GIT config with.
 - TRAIN in date:  1000/01/01
 - INFERENCE in date: 2021/06/08 and ModelVersion to score with: 1 (0=latest)


In [ ]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# Dataset access

## IN 2 SILVER - ds01_diabetes

In [10]:
from azureml.core.dataset import Dataset

# IN 2 SILVER - ds01_diabetes
## IN-data (.csv):projects/project002/11_diabetes_model_reg/train/ds01_diabetes/in/dev/1000/01/01/*.csv
## OUT: Wrote prepped data to /mnt/azureml/cr/j/4d4ec72940434242863242e8615d1f06/cap/data-capability/wd/M11_ds01_diabetes_train_SILVER/silver.parquet

ds02 = Dataset.get_by_name(workspace=p.ws, name='M11_ds01_diabetes_train_SILVER',  version='latest')
ds02.to_pandas_dataframe().head()


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [14]:
input_path_parquet = 'projects/project002/11_diabetes_model_reg/train/ds01_diabetes/in/dev/1000/01/01/*.csv' # 1_latest, 1_latest/, 1_latest/*.parquet, 1_latest/gold.parquet
ds03 = Dataset.Tabular.from_delimited_files(path = [(p.datastore, input_path_parquet)])
ds03.to_pandas_dataframe().head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


## Dataset: TRAIN - SILVER MERGED 2 GOLD

In [29]:
from azureml.core.dataset import Dataset
# SILVER MERGED 2 GOLD

# IN-data: M11_GOLD
## projects/project002/11_diabetes_model_reg/train/ds01_diabetes/out/silver/dev/b6a52bcc-ec45-49e5-8d68-022a10d1a023/
##OUT: 
### Local Path from run.output_datasets[output_to_score_gold_name]: /mnt/azureml/cr/j/7387d4a5211a414a91fc07eb3e34f436/cap/data-capability/wd/M11_GOLD/gold.parquet
#### projects/project002/11_diabetes_model_reg/train/gold/dev/1_latest/gold.parquet

input_path_parquet = 'projects/project002/11_diabetes_model_reg/train/gold/dev/1_latest/' # 1_latest, 1_latest/, 1_latest/*.parquet, 1_latest/gold.parquet
aml_dataset_in = Dataset.Tabular.from_parquet_files(path = [(p.datastore, input_path_parquet)])
aml_dataset_in.to_pandas_dataframe().head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [30]:
aml_dataset_in

{
  "source": [
    "('project002prjsp', 'projects/project002/11_diabetes_model_reg/train/gold/dev/1_latest/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ]
}

In [31]:
ds06 = Dataset.get_by_name(workspace=p.ws, name='M11_GOLD',  version='latest')
ds06.to_pandas_dataframe().head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [32]:
ds06

{
  "source": [
    "('project002prjsp', 'projects/project002/11_diabetes_model_reg/train/gold/dev/1_latest/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "3401dbd3-d8ac-43e3-b6db-e7b62809f71a",
    "name": "M11_GOLD",
    "version": 32,
    "workspace": "Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG')"
  }
}

## Dataset: TRAIN - SPLIT and REGISTER datasets (train, test, validate)

In [18]:
from azureml.core.dataset import Dataset

input_path_parquet = 'projects/project002/11_diabetes_model_reg/train/gold/dev/1_latest/' # 1_latest, 1_latest/, 1_latest/*.parquet, 1_latest/gold.parquet
aml_dataset_in = Dataset.Tabular.from_parquet_files(path = [(p.datastore, input_path_parquet)])
aml_dataset_in.to_pandas_dataframe().head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [37]:
ds04 = Dataset.get_by_name(workspace=p.ws, name='M11_GOLD_TRAIN',  version='37') 
ds04.to_pandas_dataframe().head()
ds04
# M11_GOLD_TRAIN/gold_train.parquet

# M11_GOLD_TRAINgold_train.parquet '
##! Inget slash
##! Ett space efter ,parquet

{
  "source": [
    "('project002prjsp', 'projects/project002/11_diabetes_model_reg/train/gold/dev/Train/{id_folder}/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "28b4571d-3b9b-499b-9d70-f5f491c0ed4e",
    "name": "M11_GOLD_TRAIN",
    "version": 37,
    "workspace": "Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG')"
  }
}

In [41]:
ds05 = Dataset.get_by_name(workspace=p.ws, name='M11_GOLD_TRAIN',  version='latest') 
#ds05.to_pandas_dataframe().head()
ds05

{
  "source": [
    "('project002prjsp', 'projects/project002/11_diabetes_model_reg/train/gold/dev/Train/{id_folder}/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "28b4571d-3b9b-499b-9d70-f5f491c0ed4e",
    "name": "M11_GOLD_TRAIN",
    "version": 37,
    "workspace": "Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG')"
  }
}

In [42]:
ds05.to_pandas_dataframe().head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.045341,-0.044642,-0.006206,-0.015999,0.125019,0.125198,0.019187,0.034309,0.032433,-0.005220,219.0
1,0.092564,-0.044642,0.036907,0.021872,-0.024960,-0.016658,0.000779,-0.039493,-0.022512,-0.021788,70.0
2,0.063504,0.050680,-0.004050,-0.012556,0.103003,0.048790,0.056003,-0.002592,0.084495,-0.017646,202.0
3,0.096197,-0.044642,0.051996,0.079254,0.054845,0.036577,-0.076536,0.141322,0.098646,0.061054,230.0
4,0.012648,0.050680,-0.020218,-0.002228,0.038334,0.053174,-0.006584,0.034309,-0.005145,-0.009362,111.0


In [47]:
#https://msftweudevcmnai2.blob.core.windows.net/lake3/projects/project002/11_diabetes_model_reg/train/active/gold_trained_runinfo
ds06 = Dataset.get_by_name(workspace=p.ws, name='M11_GOLD_TRAINED_RUNINFO',  version='latest') 

pd.set_option("max_colwidth", 500)
#pd.options.display.max_rows = 4000

ds06.to_pandas_dataframe()['training_data_used']

0    projects/project002/11_diabetes_model_reg/train/gold/dev/1000/01/01/bf94264b-7012-4370-ad71-6c2af3a2babf/
Name: training_data_used, dtype: object

In [49]:
ds07 = Dataset.get_by_name(workspace=p.ws, name='M11_GOLD_TRAIN',  version='latest') 
ds07.name

'M11_GOLD_TRAIN'

### DATASET: See Run info from last gold scored

In [ ]:
from azureml.core.dataset import Dataset
# p = ESMLProject()
# ...
# ...p.connect_to_lake()
meta_dataset = Dataset.get_by_name(p.ws, name="M10_GOLD_SCORED_RUNINFO")
meta_dataset.to_pandas_dataframe()

## PIPELINE - Get earlier published pipeline - run WITHOUT caching
- withour reuse, meaning regenerate_outputs=True

In [ ]:
# Get and run Pipeline - FETCH & RUN - without CACHING, e.g. regenerate=True
from azureml.pipeline.core import PublishedPipeline

# Azure ML start
p.ws = p.get_workspace_from_config()
pipeline1 = PublishedPipeline.get(workspace=p.ws, id="cb0be1c9-0df0-4821-8d2a-2d734a6b9dcb") # published_pipeline_id
# Azure ML end

# ESML (wraps Azure ml) start
pipeline_run = p_factory.execute_pipeline(pipeline1,esml_pipeline_types.IN_2_GOLD_SCORING,None,True) # execute_pipeline(self, pipeline = None, pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING, pipeline_parameters = None, regenerate_outputs=True): 
pipeline_run.wait_for_completion(show_output=False)
# ESML end

## Get old RUN, such as AutoMLRun

In [1]:
from azureml.train.automl.run import AutoMLRun

automl_step_run_id = 'AutoML_d1093aff-84fe-45d8-939f-effe3092dd4c_0' # Look at the Experiment/PipelineRun, raw JSON,  and get the runId of AutoMLStep/Run
experiment_run = p.ws.experiments[p.experiment_name]
best_run = AutoMLRun(experiment_run, run_id = automl_step_run_id)

run_id = best_run.id
model_name = best_run.properties['model_name'] # AutoML default name
print(model_name)

score = best_run.properties['score']
algo = best_run.properties['run_algorithm']
prep = best_run.properties['run_preprocessor']
primary_metric = best_run.properties['primary_metric']
peak_memory_usage = best_run.properties['peak_memory_usage']
num_logical_cores = best_run.properties['num_logical_cores']

print("score: {}, algorithm: {}, prep: {}, primary metric: {}, peak_memory_used {} MB, num_logical_core_used {}".format(score,algo,prep,primary_metric,peak_memory_usage,num_logical_cores))

# Init ESML Controller: and Register model from RUN

In [ ]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")

from esmlrt.interfaces.iESMLController import IESMLController
from esmlrt.interfaces.iESMLModelCompare import IESMLModelCompare
from esmlrt.interfaces.iESMLTestScoringFactory import IESMLTestScoringFactory
from esmlrt.interfaces.iESMLTrainer import IESMLTrainer
from esmlrt.runtime.ESMLController import ESMLController
from esmlrt.runtime.ESMLModelCompare2 import ESMLModelCompare
from esmlrt.runtime.ESMLTestScoringFactory2 import ESMLTestScoringFactory

ws = p.ws
target_column_name = p.active_model["label"]
esml_modelname = p.model_folder_name
esml_model_alias = p.ModelAlias
esml_current_env  = p.dev_test_prod
train_ds = p.GoldTrain
validate_ds = p.GoldValidate
test_ds = p.GoldTest

secret_name_tenant = p.security_config["tenant"]
secret_name_sp_id_name = p.security_config["kv-secret-esml-projectXXX-sp-id"]
secret_name_sp_secret_name = p.security_config["kv-secret-esml-projectXXX-sp-secret"]

ml_type = "regression" # Configure this: regression,classification
project_name = p.project_folder_name
test_scoring = ESMLTestScoringFactory(ml_type) # If custom, You need to implement IESMLTestScoringFactory
comparer = ESMLModelCompare(setting_path = "") # If custom. You need to implement IESMLModelCompare

# IESMLController: you do not have to change/implement this class. Dependency injects default or your class.
controller = ESMLController(comparer,test_scoring,project_name,esml_modelname, esml_model_alias, secret_name_tenant,secret_name_sp_id_name,secret_name_sp_secret_name)
controller.dev_test_prod = esml_current_env
controller.version()

## REGISTER from run: AutoMLRun as example (same principle for PipelineRun and Step)
- NB! For AutoML Run we get the MODEL_NAME from the child_run, but `we need the MAIN_RUN`, when registering the model

In [ ]:
from azureml.train.automl.run import AutoMLRun

# Get the MODEL_NAME from CHILD RUN
automl_step_run_id = 'AutoML_d1093aff-84fe-45d8-939f-effe3092dd4c_0' # Look at the Experiment/PipelineRun, raw JSON,  and get the runId of AutoMLStep/Run
experiment_run = p.ws.experiments[p.experiment_name]
best_run = AutoMLRun(experiment_run, run_id = automl_step_run_id)

run_id = best_run.id
model_name = best_run.properties['model_name'] # AutoML default name

# Get the MAIN_RUN to be able to pass when run.register_model()
automl_step_run_id = 'AutoML_d1093aff-84fe-45d8-939f-effe3092dd4c' # Removed the '_0' and we get the parent MAIN_RUN
experiment_run = p.ws.experiments[p.experiment_name]
main_run = AutoMLRun(experiment_run, run_id = automl_step_run_id)

main_run.properties["AMLSettingsJsonString"]  # Only the MAIN_RUN has this property

In [ ]:
#REGISTER passing the MAIN_RUN as parameter

extra_tags = {"status_code": IESMLController.esml_status_new, "trained_with": "AutoMLRun"}
model_registered_as_new = controller.register_model(source_ws=ws, target_env="dev", source_model=None, run=main_run, esml_status=IESMLController.esml_status_new,extra_model_tags=extra_tags)

# AML: MODEL: Download FITTED MODEL from Azure, locally, load & predict with it

In [ ]:
import joblib
import os
from azureml.core import Model

model = Model(p.ws,'AutoMLd1093aff80')
m = model.download(target_dir='.', exist_ok=True)

model_file_name = 'ridge_{0:.2f}.pkl'.format(0.0)
model_file_name2 = 'model.pkl'
local_path='./wd/'+model_file_name2
fitted_model = joblib.load(local_path)

## Example: trouble shooting errors in pipeline, can be done locally
- example below: If you have your own TestScoringFactory, and something is wrong.

In [ ]:
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error

validation_data_pd = p.GoldValidate.to_pandas_dataframe()
label = p.active_model['label']
y_test = validation_data_pd.pop(label).to_frame()

# Calculate root-mean-square error
y_actual = y_test.values.flatten().tolist()
y_predict = fitted_model.predict(validation_data_pd)

rmse = None
try:
    rmse = sqrt(mean_squared_error(y_actual, y_predict))
except Exception as e:
    mess = "ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U5'), dtype('float64')) -> None"
    if(mess in str(e)):
        print("asdf")
    print(type(y_actual))
    print(type(y_predict))
    y_actual = y_test.values.flatten()
    print(type(y_actual))
    print(type(y_predict))
    rmse = sqrt(mean_squared_error(y_actual, y_predict))
    print(rmse)

# AML: Azure ML - Connect to workspace, without ESML, and update to legacy_mode

In [ ]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.authentication import ServicePrincipalAuthentication

auth = ServicePrincipalAuthentication(tenant_id='todo',service_principal_id='todo', service_principal_password='todo')
config_name = "dev_ws_config.json"
ws = Workspace.get(name = 'aml-prj001-weu-dev-001',subscription_id = 'todo',resource_group = 'todo',auth=auth)
ws = Workspace.from_config(path="../../", auth=auth, _file_name=config_name)

ws.update(v1_legacy_mode=True)

## AML: Write login config cache  to disk

In [ ]:
import os
old_loc = os.getcwd()
try:
    ws.write_config(path="../../", file_name=config_name)
except Exception as e:
    raise Exception("Error - authenticate_workspace_and_write_config - Cannot write config-file") from e
finally: 
    os.chdir(old_loc) # Change back to working location...

### AML: Verify that we can read Login cache to retrieve Auzre ML workspace, without Auth

In [ ]:
ws = Workspace.from_config(path="../../", auth=auth, _file_name=config_name)

# AML Pipeline RUN: Dehydrate RUN (from IN_2_GOLD_TRAIN_MANUAL

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
from azureml.pipeline.core import PipelineRun
from azureml.train.automl.run import AutoMLRun
from azureml.core import Run

main_pipeline_run_id = '1c824e67-13c8-44dd-932c-57be815ddf93'
manual_train_run_id= '3e015cb7-2a70-4758-b5ad-c846e181ca15'

experiment_run = p.ws.experiments[p.experiment_name]
best_run = PipelineRun(experiment_run, run_id = manual_train_run_id) # azureml.StepRun

run_id = best_run.id
print(run_id)

### Init ESMLController, to be able to register model easier, with correct TAGS & MLOps inner/outer loop support

In [ ]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")
from esmlfac.adapter import ESMLFactory

# Init a ESMLController from ESMLProject configuration: Needed for 
datastore = p.connect_to_lake() # Connects to the correct ALDS GEN 2 storage account (DEV, TEST or PROD)
controller = ESMLFactory.get_esml_controller_from_notebook(p)

### Register model - model_path overrides name in Azure ML
- https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py#azureml-core-run-register-model

In [ ]:
model_name_aml = 'AMLModeName'# 'AutoMLd1093aff80'
model_path = 'outputs/ridge_0.00.pkl' #important that path is correct, should exists in PipelieStep, not in Pipeline
m = Model(p.ws,model_name_aml)
controller.register_model(source_model=m,run=best_run, source_ws=p.ws,target_env="dev",esml_status='esml_promoted_2_dev',model_path=model_path)

### Verify that the model also can be downloaded & initiated

# 1b) DEHYDRATE Model, from IN_2_GOLD_TRAIN_MANUAL

In [ ]:
import os
import joblib

wrong_model_path = 'outputs_or_other/kalle.pkl' # does not matter what path, it is the local target
m_path = m.download(target_dir=wrong_model_path, exist_ok=True)
fitted_model = joblib.load(m_path)
type(fitted_model)

### 1c) Test using the fitted model - predict, calculate test scoring

In [ ]:
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error,r2_score

validation_data_pd = p.GoldValidate.to_pandas_dataframe()
label = p.active_model['label']
y_test = validation_data_pd.pop(label).to_frame()

# Calculate root-mean-square error
y_actual = y_test.values.flatten().tolist()
y_predict = fitted_model.predict(validation_data_pd)

rmse = None
try:
    rmse = sqrt(mean_squared_error(y_actual, y_predict))
    print(rmse)
except Exception as e:
    mess = "ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U5'), dtype('float64')) -> None"
    if(mess in str(e)):
        print("asdf")
    print(type(y_actual))
    print(type(y_predict))
    y_actual = y_test.values.flatten()
    print(type(y_actual))
    print(type(y_predict))
    rmse = sqrt(mean_squared_error(y_actual, y_predict))
    print(rmse)

# AML: Azure ML - Connect to workspace, without ESML, and update to legacy_mode

In [ ]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.authentication import ServicePrincipalAuthentication

auth = ServicePrincipalAuthentication(tenant_id='todo',service_principal_id='todo', service_principal_password='todo')
config_name = "dev_ws_config.json"
ws = Workspace.get(name = 'aml-prj001-weu-dev-001',subscription_id = 'todo',resource_group = 'todo',auth=auth)
ws = Workspace.from_config(path="../../", auth=auth, _file_name=config_name)

ws.update(v1_legacy_mode=True)

## AML: Write login config cache  to disk

In [ ]:
import os
old_loc = os.getcwd()
try:
    ws.write_config(path="../../", file_name=config_name)
except Exception as e:
    raise Exception("Error - authenticate_workspace_and_write_config - Cannot write config-file") from e
finally: 
    os.chdir(old_loc) # Change back to working location...

### AML: Verify that we can read Login cache to retrieve Auzre ML workspace, without Auth

In [ ]:
ws = Workspace.from_config(path="../../", auth=auth, _file_name=config_name)

# Attach Databricks cluster to AML workspace

In [ ]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute import DatabricksCompute

dev_rg = 'abc-def-esml-project001-weu-dev-001-rg'
dev_workspace = 'dbx-prj001-weu-dev-001'
db_compute_name = 's-p001-m01-rt91' # Your pre-created DBX cluster - friendlu name
my_dbx_cluster_url_name = '1234-1234-zxcvzxv' # See web browser URL when having cluster selected

db_resource_group = 'abc-def-esml-project001-weu-dev-004-rg'
db_workspace_name = 'dbx-prj001-weu-dev-003'

all_envs = {
    'dev': {'resource_group': dev_rg, 'workspace_name':dev_workspace},
    'test': {'resource_group': 'xyz-xyz-esml-project001-weu-test-001-rg', 'workspace_name': 'z'},
    'prod': {'resource_group': 'xyz-xyz-esml-project001-weu-prod-001-rg', 'workspace_name': 'z'}
}
all_envs = p_factory.set_dbx_token(all_envs)
db_access_token=all_envs[p.dev_test_prod]['access_token']  # Databricks access token, is earlier fetched from keyvault

# If you get the below - the token is expired or faulty.
'''
Service invocation failed!
Request: GET adb-1234.11.azuredatabricks.net/api/2.0/workspace/list
Status Code: 403 Forbidden
Reason Phrase: Forbidden
Response Body: {"error_code":"403","message":"Invalid access to Org: 1234"}

'''

config = DatabricksCompute.attach_configuration(
resource_group = db_resource_group,
workspace_name = db_workspace_name,
access_token= db_access_token)

databricks_compute=ComputeTarget.attach(p.ws, db_compute_name, config)